In [1]:
# repo root
%cd /work

/work


In [2]:
!ls 

LICENSE    adapters  examples  paper	       scripts	tests
README.md  docker    out       pyproject.toml  src


In [3]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [4]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [5]:
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cards)


In [6]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [7]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
 [100%] Downloaded 33730584 bytes...
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [20]:
# Fig2 run
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --taus 0.2,0.4,0.6,0.8,0.9

[run_fig2] sample_card note (HNSC) not found: /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json


In [21]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours

audit_log.tsv		    report.jsonl	       tau_0.40
claims.proposed.tsv	    report.md		       tau_0.60
distilled.tsv		    risk_coverage.tsv	       tau_0.80
distilled.with_modules.tsv  run_meta.json	       tau_0.90
evidence.normalized.tsv     sample_card.resolved.json  term_gene_edges.tsv
modules.tsv		    tau_0.20		       term_modules.tsv


In [22]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.20
report   True 3285
audit    True 3361
modules  True 13767
edges    True 88967
terms    True 3188
distilled True 37821
risk     True 235
card     True 414
meta     True 3487

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (3, 30)
[report] decisions: {'PASS': 3}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'cancer', 'comparison', 'tau', 'schema_version', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.disease', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'evidence_refs.gene_set_hash', 'benchmark_id', 'claim.claim_id', 'claim.entity', 'claim.direction', 'claim.context_keys', 'claim.evidence_ref.module_id', 'claim.evidence_ref.gene_ids']

[report] contract view (head):
schema_version   benchmark_id                run_id method cancer          comp

In [23]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80
report   True 3285
audit    True 3361
modules  True 13767
edges    True 88967
terms    True 3188
distilled True 37821
risk     True 235
card     True 414
meta     True 3487

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (3, 30)
[report] decisions: {'PASS': 3}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'cancer', 'comparison', 'tau', 'schema_version', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.disease', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'evidence_refs.gene_set_hash', 'benchmark_id', 'claim.claim_id', 'claim.entity', 'claim.direction', 'claim.context_keys', 'claim.evidence_ref.module_id', 'claim.evidence_ref.gene_ids']

[report] contract view (head):
schema_version   benchmark_id                run_id method cancer          comp

In [29]:
import pandas as pd
p="paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/tau_0.80/distilled.tsv"
df=pd.read_csv(p, sep="\t")
print("n", len(df), "cols", list(df.columns)[:20])
if "term_survival" in df.columns:
    s=pd.to_numeric(df["term_survival"], errors="coerce")
    print("term_survival describe:\n", s.describe())
    print("unique (head):", sorted(s.dropna().unique())[:20])
else:
    print("NO term_survival column")

n 50 cols ['term_id', 'term_name', 'source', 'stat', 'qval', 'direction', 'evidence_genes', 'pval', 'is_summary', 'group_id', 'evidence_genes_str', 'qval_source', 'qval_provided', 'pval_provided', 'is_valid', 'invalid_reason', 'n_evidence_genes', 'masked_genes_count', 'term_row_id', 'term_uid']
term_survival describe:
 count    50.000000
mean      0.933438
std       0.063513
min       0.734375
25%       0.906250
50%       0.953125
75%       0.984375
max       1.000000
Name: term_survival, dtype: float64
unique (head): [np.float64(0.734375), np.float64(0.78125), np.float64(0.8125), np.float64(0.828125), np.float64(0.875), np.float64(0.890625), np.float64(0.90625), np.float64(0.921875), np.float64(0.9375), np.float64(0.953125), np.float64(0.96875), np.float64(0.984375), np.float64(1.0)]


In [31]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [43]:
!python paper/scripts/run_fig2_pipeline.py \
    --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
    --variants ours,shuffled_context \
    --taus 0.2,0.4,0.6,0.8,0.9

[run_fig2] OK
  out: /work/paper/source_data/PANCAN_TP53_v1/out


In [42]:
!python paper/scripts/collect_fig2_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --benchmark-id PANCAN_TP53_v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv rows=70


In [45]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/out/Fig2.risk_coverage.panel.pdf \
  --benchmark-id PANCAN_TP53_v1 \
  --cols 4 \
  --fontsize 16 \
  --title "Risk–coverage curves across cancers"
